In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import librosa
import os
import sys


In [3]:
# main_path = '/content/drive/MyDrive/baby_cry'
# data_path = os.path.join(main_path, 'data2')

# AUDIO_DIR = os.path.join(main_path, 'audios')
# MODEL_PATH = os.path.join(main_path, 'coAtNet.pt')

In [10]:
main_path = '/Users/jaewone/developer/tensorflow/baby-cry-classification'
data_path = os.path.join(main_path, 'data')

work_path = os.path.join(main_path, 'model', 'coAtNet')
AUDIO_DIR = os.path.join(work_path, 'audios2')
MODEL_PATH = os.path.join(work_path, 'model.pt')

sys.path.append(main_path)

In [16]:
# # copy file
# from shutil import copyfile
# from typing import Union, Optional
# from random import sample, seed, choices, shuffle
# from math import isclose

# def copy_file(src: str, dst: str):
#     if os.path.isfile(src):
#         copyfile(src, dst)
#     else:
#         raise OSError(f'{src} is not a file')


# # 각각의 state에서 n개의 무작위 파일을 선택하여 경로를 반환한다.
# def get_state_samples(data_path: str,
#                       state_list: Optional[list[str]] = None,
#                       n_extract: int = 100,
#                       rand_seed: int = 123) -> list[str]:
#     """
#     각각의 state에서 n개의 무작위 파일을 선택하여 경로를 반환한다.

#     Parameters:

#         * data_path : 파일의 경로

#         * state_list=None : state 리스트를 받을 경우 state_list가 포함하는 state 폴더의 파일들만 이름을 변경한다.

#         * n_extract=100 : 각각의 state 별 추출할 파일 개수. 만약 n_etract 보다 파일 개수가 부족하다면 파일의 개수 만큼만 추출한다.

#         * rand_seed=123 : 난수를 생성하는 시드값으로 동일한 시드값은 동일한 결과를 보장한다.

#     Returns: 랜덤 추출한 파일의 경로 리스트
#     """

#     if not os.path.exists(data_path):
#         raise OSError(f'path {data_path} not exist.')

#     state_list = ['sleepy', 'uncomfortable', 'diaper', 'awake', 'sad', 'hug', 'hungry']

#     seed(rand_seed)
#     file_list = []
#     for state in state_list:
#         state_file_list = [os.path.join(data_path, state, file) for file in os.listdir(
#             os.path.join(data_path, state))]
#         file_list = file_list + (sample(state_file_list, k=n_extract)
#                                  if len(state_file_list) > n_extract else state_file_list)

#     return file_list

# # 각각의 state에서 n개의 무작위 파일을 추출한다.
# def extract_state_sample(data_path: str,
#                          output_dir: str,
#                          n_extract: int = 100,
#                          rand_seed: int = 123,
#                          state_list: Optional[list[str]] = None,
#                          with_dir: bool = True) -> list[str]:
#     """
#     각각의 state에서 n개의 무작위 파일을 추출한다.

#     Parameters:

#         * data_path : 파일의 경로

#         * output_dir : sample 파일들을 저장할 폴더 경로. 만약 경로가 이미 존재 할 경우 예외를 발생시키며 경로가 없으면 생성한다.

#         * n_extract=100 : 각각의 state 별 추출할 파일 개수. 만약 n_etract 보다 파일 개수가 부족하다면 파일의 개수 만큼만 추출한다.

#         * rand_seed=123 : 난수를 생성하는 시드값으로 동일한 시드값은 동일한 결과를 보장한다.

#         * state_list=None : state 리스트를 받을 경우 state_list가 포함하는 state 폴더의 파일들만 이름을 변경한다.

#         * with_dir=True : False일 경우 output_path에 state에 따른 폴더 구분 없이 데이터를 추출한다.

#     Returns: 랜덤 추출하여 저장된 파일의 경로 리스트
#     """

#     # Check output_dir exist.
#     if os.path.exists(output_dir):
#         raise OSError(f'path {output_dir} already exist.')
#     else:
#         os.mkdir(output_dir)

#     state_list = ['sleepy', 'uncomfortable', 'diaper', 'awake', 'sad', 'hug', 'hungry']

#     # Create folder of state if with_dir is True
#     if with_dir:
#         for state in state_list:
#             os.mkdir(os.path.join(output_dir, state))

#     # Extract n sample of state
#     sample_data_list = get_state_samples(
#         data_path, state_list, n_extract, rand_seed)

#     # Copy files with sample data list
#     output_file_list = []
#     for file in sample_data_list:
#         if with_dir:
#             _, dir, filename = file.rsplit('/', 2)
#             output_file = os.path.join(output_dir, dir, filename)
#             output_file_list.append(output_file)
#             copy_file(file, output_file)
#         else:
#             output_file = os.path.join(output_dir, file.rsplit('/', 1)[1])
#             output_file_list.append(output_file)
#             copy_file(file, output_file)

#     return output_file_list


In [3]:
from trans_data import extract_state_sample, get_state_file_list

if not os.path.exists(AUDIO_DIR):
    extract_state_sample(data_path, AUDIO_DIR, 10, with_dir=True)

In [4]:
torch.cuda.is_available()

False

In [5]:
class CoAtNet(nn.Module):
    def __init__(self, num_classes=36):
        super(CoAtNet, self).__init__()

        # Convolutional part
        self.conv_layers = nn.Sequential(
            # in_channels = 1 : Number of channels in the input image
            # out_channels = 32 : Number of channels produced by the convolution
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Transformer part
        encoder_layer = nn.TransformerEncoderLayer(d_model=32, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, num_layers=2)

        # Linear classifier
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.conv_layers(x)

        # Flattening
        x = x.view(x.size(0), -1, x.size(1))

        # Transformer encoder
        x = self.transformer_encoder(x)

        # Max pooling over time
        x, _ = torch.max(x, dim=1)

        # Classifier
        x = self.fc(x)
        return x


In [7]:
file_list = get_state_file_list(AUDIO_DIR)
print(file_list[0])
len(file_list)

/Users/jaewone/developer/tensorflow/baby-cry-classification/model/coAtNet/audios2/sad/sad_326.wav


70

In [8]:
class ToMelSpectrogram:
    def __call__(self, samples):
        # print(samples.shape) # (16000,)
        mel_spectrogram = librosa.feature.melspectrogram(y=samples, sr=16000, n_mels=64, hop_length=225)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
        return log_mel_spectrogram

class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        self.class_map = {'sleepy': 0, 'uncomfortable': 1, 'diaper': 2, 'awake': 3, 'sad': 4, 'hug': 5, 'hungry': 6}
        self.label_list = np.array([self.class_mapping(file.rsplit('/', 2)[1]) for file in file_list])

    def class_mapping(self, class_name:str) -> int:
        return self.class_map[class_name]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        waveform, _ = librosa.load(self.file_list[idx],
                                   sr=None,
                                   duration=1.0,
                                   mono=True)
        label = self.label_list[idx]

        if self.transform:
            waveform = self.transform(waveform)

        return waveform, label



In [27]:
# class ToMelSpectrogram:
#     def __call__(self, samples):
#         # print(samples.shape) # (16000,)
#         return librosa.feature.melspectrogram(y=samples, sr=16000, n_mels=64, hop_length=225)

# class AudioDataset(torch.utils.data.Dataset):
#     def __init__(self, data_dir, transform=None):
#         self.data_dir = data_dir
#         self.transform = transform
#         self.file_list = os.listdir(self.data_dir)
#         self.class_map = {'sleepy': 0, 'uncomfortable': 1, 'diaper': 2, 'awake': 3, 'sad': 4, 'hug': 5, 'hungry': 6}
#         self.label_list = np.array([self.class_mapping(file.split('_')[0]) for file in file_list])

#     def class_mapping(self, class_name:str) -> int:
#         return self.class_map[class_name]

#     def __len__(self):
#         return len(self.file_list)

#     def __getitem__(self, idx):
#         waveform, _ = librosa.load(os.path.join(self.data_dir, self.file_list[idx]),
#                                    sr=None,
#                                    duration=1.0,
#                                    mono=True)
#         label = self.label_list[idx]

#         if self.transform:
#             waveform = self.transform(waveform)

#         return waveform, label



In [9]:
train_file_list, val_file_list = train_test_split(file_list, test_size=0.3, stratify=[file.rsplit('/', 2)[1] for file in file_list])
val_file_list, test_file_list = train_test_split(val_file_list, test_size=0.33, stratify=[file.rsplit('/', 2)[1] for file in val_file_list])

print(len(train_file_list))
print(len(val_file_list))
print(len(test_file_list))


with open(os.path.join(work_path, "test_file_list.txt"), "w") as txt_file:
    # txt_file.write(str([file.rsplit('/', 1)[1] for file in test_file_list]))
    txt_file.write(str(test_file_list))

49
14
7


In [ ]:
def train():
    # We will use the transformation to convert the audio into Mel spectrogram
    transform = Compose([ToMelSpectrogram(), ToTensor()])

    train_set = AudioDataset(train_file_list, transform=transform)
    val_set = AudioDataset(val_file_list, transform=transform)

    train_loader = DataLoader(dataset=train_set, batch_size=16, shuffle=True)
    val_loader = DataLoader(dataset=val_set, batch_size=16, shuffle=True)

    # Assuming we have this class implemented following the paper or using a library
    model = CoAtNet()
    model = model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    criterion = nn.CrossEntropyLoss().cuda()

    num_epochs = 10

    for epoch in range(num_epochs):
        model.train()
        for data in train_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()  # Move both tensors to GPU


            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

        # Validation
        if (epoch + 1) % 5 == 0:
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for inputs, labels in val_loader:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print(f"Validation Accuracy: {correct/total}")

    torch.save(model.state_dict(), MODEL_PATH)

# train()

In [ ]:
transform = Compose([ToMelSpectrogram(), ToTensor()])
test_set = AudioDataset(test_file_list, transform=transform)
test_loader = DataLoader(dataset=test_set, batch_size=16, shuffle=True)

In [ ]:
class PredictDataset(torch.utils.data.Dataset):
    def __init__(self, audio_paths, transform=None):
        self.audio_paths = audio_paths
        self.transform = transform

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        audio_path = self.audio_paths[idx]
        audio_clip = load_audio_clip(audio_path)

        if self.transform:
            audio_clip = self.transform(audio_clip)

        return audio_clip


def load_model(path):
    model = CoAtNet()  # should match the architecture of the trained model
    model.load_state_dict(torch.load(path))
    model.eval()
    return model


def predict(audio_paths):
    model = load_model(MODEL_PATH)

    transform = transforms.Compose([
        # add transforms that were used while training
    ])

    dataset = PredictDataset(audio_paths, transform=transform)
    data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

    predictions = []

    for batch in data_loader:
        batch = batch.cuda()
        outputs = model(batch)
        # change if multi-label classification
        _, predicted = torch.max(outputs.data, 1)
        predictions.append(predicted.item())

    return predictions


def main():
    audio_paths = ["audio1.wav", "audio2.wav",
                   "audio3.wav"]  # replace with actual paths
    predictions = predict(audio_paths)
    print(predictions)


if __name__ == "__main__":
    main()
